# Project 3

### Reading files

In [1]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import os 
import scipy
from scipy import signal as sig
import random
from scipy.linalg import svd
cast_path = "Data/cast"
cone_path = "Data/cone"

# path = input('Enter 1 for DanaHall\nEnter 2 for DanaOffice\n')
path = 2
if path==1:
    path = cast_path
else:
    path = cone_path
    

file_li = []    
for file in os.listdir(path):
    file_li.append(file)
    
    
    


## Compute Harris and Non Max Supression

In [ ]:
def sobel_filter(img, axis):
    kernel = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    if axis == 'x':
        kernel = np.transpose(kernel)
    filtered = np.zeros_like(img, dtype=np.float32)
    img = np.pad(img, 1, mode='edge')
    for i in range(1, img.shape[0]-1):
        for j in range(1, img.shape[1]-1):
            filtered[i-1, j-1] = np.sum(kernel * img[i-1:i+2, j-1:j+2])
    return filtered

def filter_img(image, kernel):
    # Get kernel dimensions
    k_height, k_width = kernel.shape

    k_pad = k_height//2
    
#     image = cv2.copyMakeBorder(image, k_pad, k_pad, k_pad, k_pad, cv2.BORDER_CONSTANT, value=0)
    
    # Get image dimensions
    img_height, img_width = image.shape

    # Create output image
    output = np.zeros_like(image)

    # Loop over image pixels
    for y in range(k_height//2, img_height - k_height//2):
        for x in range(k_width//2, img_width - k_width//2):
            # Get the region of interest
            roi = image[y-k_height//2:y+k_height//2+1, x-k_width//2:x+k_width//2+1]

            # Apply the kernel to the ROI
            filtered_pixel = (roi * kernel).sum()

            # Set the output pixel value
            output[y, x] = filtered_pixel

    return output 

def compute_harris_r(img, ksize, k):

    # Compute gradients in x and y direction
    dx = sobel_filter(img, axis='x')
    dy = sobel_filter(img, axis='y')
    
#     cv2.imshow('dx',dx)
#     cv2.imshow('dy',dy)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

    # Compute products of gradients at each pixel
    dx2 = dx ** 2
    dy2 = dy ** 2
    dxy = dx * dy

    # Compute sum of products of gradients in local window
    w = np.ones((ksize, ksize), np.float64)
    sdx2 = filter_img(dx2, w)
    sdy2 = filter_img(dy2, w)
    sdxy = filter_img(dxy, w)
    
#     cv2.imshow('sdx2',sdx2)
#     cv2.imshow('sdy2',sdy2)

#     cv2.imshow('dxy',dxy)

#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

    # Compute Harris R function
    det = sdx2 * sdy2 - sdxy ** 2
    trace = sdx2 + sdy2
    r = det - k * trace ** 2
    
    
    return r

def non_max_suppression(img, size, threshold):
    # Find local maxima above threshold
    h, w = img.shape
    maxima = []
    w_o_nms = []
    for y in range(size, h - size):
        for x in range(size, w - size):
            w_o_nms.append((x,y))
            if img[y, x] > threshold and img[y, x] == np.max(img[y - size:y + size + 1, x - size:x + size + 1]):
                maxima.append((x, y))

    return maxima,w_o_nms

## Find and Plot correspondence

In [ ]:
def find_correspondences(img1, corners1, img2, corners2, patch_size, ncc_threshold):
    correspondences = []
    for i in range(len(corners1)):
        max_ncc = -1
        best_match = None
        for j in range(len(corners2)):
            # Extract patches centered at corners
            patch1 = img1[corners1[i][1]-patch_size//2 : corners1[i][1]+patch_size//2,
                          corners1[i][0]-patch_size//2 : corners1[i][0]+patch_size//2]
            patch2 = img2[corners2[j][1]-patch_size//2 : corners2[j][1]+patch_size//2,
                          corners2[j][0]-patch_size//2 : corners2[j][0]+patch_size//2]
            
#             print(patch1)
            # Compute normalized cross correlation (NCC)
            ncc = np.sum((patch1 - np.mean(patch1)) * (patch2 - np.mean(patch2))) / \
                  (np.sqrt(np.sum((patch1 - np.mean(patch1)) ** 2)) * np.sqrt(np.sum((patch2 - np.mean(patch2)) ** 2)))
            # Update best match if NCC is higher than threshold and previous matches
            if ncc > max_ncc and ncc > ncc_threshold:
                max_ncc = ncc
                best_match = (i, j)
        if best_match is not None:
            correspondences.append(best_match)
    return correspondences

# correspondences = match_corners(img1, img2, corners1, corners2)
# plot_correspondences(img1, img2, correspondences)
def plot_correspondences(img1, corners1, img2, corners2, matches, title='Correspondences'):
    # Create a new image with both images side by side
    h, w = max(img1.shape[0], img2.shape[0]), img1.shape[1] + img2.shape[1]
    new_img = np.zeros((h, w, 3), dtype=np.uint8)
    new_img[:img1.shape[0], :img1.shape[1]] = img1#cv2.cvtColor(img1, cv2.COLOR_GRAY2RGB)
    new_img[:img2.shape[0], img1.shape[1]:] = img2#cv2.cvtColor(img2, cv2.COLOR_GRAY2RGB)
    cv2.imshow(title, new_img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

    # Draw lines between matching corners
    for i, match in enumerate(matches):
        pt1 = tuple(corners1[match[0]])
        pt2 = (corners2[match[1]][0]+img1.shape[1],corners2[match[1]][1]+ 0)
        color = tuple(np.random.randint(0, 256, 3).tolist())
#         cv2.circle(new_img, corners1[i], 5, color, 2)
#         cv2.circle(new_img, (corners2[i][0]+img1.shape[1],corners2[i][1]), 5, color, 2)

#         print(pt1)
#         print(pt2)
        cv2.line(new_img, pt1, pt2, color, 2)

    
    # Display the image with the correspondences
    cv2.imshow(title, new_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def plot_correspondences_ransac(img1, corners1, img2, corners2, title='Correspondences'):
    # Create a new image with both images side by side
    h, w = max(img1.shape[0], img2.shape[0]), img1.shape[1] + img2.shape[1]
    new_img = np.zeros((h, w, 3), dtype=np.uint8)
    new_img[:img1.shape[0], :img1.shape[1]] = img1#cv2.cvtColor(img1, cv2.COLOR_GRAY2RGB)
    new_img[:img2.shape[0], img1.shape[1]:] = img2#cv2.cvtColor(img2, cv2.COLOR_GRAY2RGB)
    cv2.imshow(title, new_img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

    # Draw lines between matching corners
    for i in range(len(corners1)):
        pt1 = corners1[i]
        pt2 = (corners2[i][0]+img1.shape[1],corners2[i][1])
#         print(pt1,pt2)
        color = tuple(np.random.randint(0, 256, 3).tolist())
        cv2.circle(new_img, corners1[i], 5, color, 2)
        cv2.circle(new_img, (corners2[i][0]+img1.shape[1],corners2[i][1]), 5, color, 2)
#         print(pt1)
#         print(pt2)
        cv2.line(new_img, pt1, pt2, color, 2)
    
    
    # Display the image with the correspondences
    cv2.imshow(title, new_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

### Computing the corners and getting matches

In [ ]:
scale_factor = 1


img1 = cv2.imread(f'{path}/'+file_li[0])
img2 = cv2.imread(f'{path}/'+file_li[1])

img1 = cv2.resize(img1,None,fx=scale_factor,fy=scale_factor,interpolation=cv2.INTER_LINEAR)
img2 = cv2.resize(img2,None,fx=scale_factor,fy=scale_factor,interpolation=cv2.INTER_LINEAR)

r1 = compute_harris_r(cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY), ksize=3, k=0.04)
corners1,c1_wo = non_max_suppression(r1, size=5, threshold=0.01*r1.max())

r2 = compute_harris_r(cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY), ksize=3, k=0.04)
corners2,c2_wo = non_max_suppression(r2, size=5, threshold=0.01*r2.max())

# Draw corners on the images
#     img1_corners = cv2.cvtColor(img1, cv2.COLOR_GRAY2RGB)
img1_corners = img1.copy()
for corner in c1_wo:
    cv2.circle(img1_corners, corner, 5, (0, 0, 255), 2)
cv2.imshow('Image 1 Corners befor non max suppression', img1_corners)

img1_corners = img1.copy()
for corner in corners1:
    cv2.circle(img1_corners, corner, 5, (0, 0, 255), 2)

#     img2_corners = cv2.cvtColor(img2, cv2.COLOR_GRAY2RGB)
img2_corners = img2.copy()

for corner in corners2:
    cv2.circle(img2_corners, corner, 5, (0, 0, 255), 2)

# Display images with corners
cv2.imshow('Image 1 Corners', img1_corners)
cv2.imshow('Image 2 Corners', img2_corners)
cv2.waitKey(0)
cv2.destroyAllWindows()



print('corners:',len(corners1),len(corners2))
patch_size = 10
ncc_threshold = 0.90
matches = find_correspondences(cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY), corners1, cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY), corners2, patch_size, ncc_threshold)

plot_correspondences(img1, corners1, img2, corners2, matches, title='Correspondences')    


## Computing F and epipolar lines

In [2]:
def normalize_points(pts):

    mean_ =np.mean(pts,axis=0)

    #finding centre
    u = pts[:,0] - mean_[0]
    v = pts[:,1] - mean_[1]

    sd_u = 1/np.std(pts[:,0])
    sd_v = 1/np.std(pts[:,1])
    Tscale = np.array([[sd_u,0,0],[0,sd_v,0],[0,0,1]])
    Ta = np.array([[1,0,-mean_[0]],[0,1,-mean_[1]],[0,0,1]])
    T = np.dot(Tscale,Ta)

    pt = np.column_stack((pts,np.ones(len(pts))))
    norm_pts = (np.dot(T,pt.T)).T

    return norm_pts,T

def estimate_F(img1_pts,img2_pts):

    #normalize points
    img1_pts,T1 = normalize_points(img1_pts)
    img2_pts,T2 = normalize_points(img2_pts)

    x1 = img1_pts[:,0]
    y1 = img1_pts[:,1]
    x1dash = img2_pts[:,0]
    y1dash = img2_pts[:,1]
    A = np.zeros((len(x1),9))

    for i in range(len(x1)):
        A[i] = np.array([x1dash[i]*x1[i],x1dash[i]*y1[i],x1dash[i], y1dash[i]*x1[i],y1dash[i]*y1[i],y1dash[i],x1[i],y1[i],1])

    #taking SVD of A for estimation of F
    U, S, V = np.linalg.svd(A,full_matrices=True)
    F_est = V[-1, :]
    F_est = F_est.reshape(3,3)

    # Enforcing rank 2 for F
    ua,sa,va = np.linalg.svd(F_est,full_matrices=True)
    sa = np.diag(sa)

    sa[2,2] = 0
   

    F = np.dot(ua,np.dot(sa,va))
    # F, mask = cv2.findFundamentalMat(img1_pts,img2_pts,cv2.FM_LMEDS)
    F = np.dot(T2.T, np.dot(F, T1))

    return F

def ransac_F(corners1,corners2,matches):
    no_iter = 1000
    threshold = 0.05
    inliers = 0
    pt1 = np.array([corners1[matches[idx][0]] for idx in range(len(matches))])
    pt2 = np.array([corners2[matches[idx][1]] for idx in range(len(matches))])
    n_rows = np.array(pt1).shape[0]

    final_indices = []
    for i in range(no_iter):
        indices = []

        #randomly select 8 points
        random = np.random.choice(n_rows,size = 8)
        img1_8pt = pt1[random]
        img2_8pt = pt2[random]
    
        F_est = estimate_F(img1_8pt,img2_8pt)

        for j in range(n_rows):
            x1 = pt1[j]
            x2 = pt2[j]

            #error computation
            pt1_ = np.array([x1[0],x1[1],1])
            pt2_ = np.array([x2[0],x2[1],1])
            error = np.dot(pt1_.T,np.dot(F_est,pt2_))
            
            if np.abs(error) < threshold:
                indices.append(j)
                
        if len(indices) > inliers:
            inliers = len(indices)
            final_indices = indices
            F = F_est

 
    img1_points = pt1[final_indices]
    img2_points = pt2[final_indices]

    F_all_inliers = estimate_F(img1_points,img2_points)

    return img1_points,img2_points, F,F_all_inliers




def draw_lines(lines_set,image,points):
    for line,pt in zip(lines_set,points):
        x0,y0 = map(int, [0, -line[2]/line[1] ])
        x1,y1 = map(int, [image.shape[1]-1, -line[2]/line[1] ])
        img = cv2.line(image, (x0,y0), (x1,y1), (0,255,0) ,2)
        img = cv2.circle(image, (int(pt[0]),int(pt[1])), 5, (0,255,0), 2)
    return img

def epipolar_lines(pts1_,pts2_,F,image1,image2):
    lines1_,lines2_ = [], []
    for i in range(len(pts1_)):
        p1 = np.array([pts1_[i,0], pts1_[i,1], 1])
        p2 = np.array([pts2_[i,0], pts2_[i,1], 1])
    
        lines1_.append(np.dot(F.T, p2))
        lines2_.append(np.dot(F,p1))
    img1 = draw_lines(lines1_,image1,pts1_)
    img2 = draw_lines(lines2_,image2,pts2_)

   

    out = np.hstack((img1, img2))
    cv2.imwrite("epipolar_lines.png",out)
    return lines1_,lines2_

In [ ]:
pts1_,pts2_, F,F_all_inliers = ransac_F(corners1,corners2,matches)
F = F_all_inliers/F_all_inliers[2,2]

lines1,lines2 = epipolar_lines(pts1_,pts2_,F,img1.copy(),img2.copy())
plot_correspondences_ransac(img1, pts1_, img2, pts2_, title='Correspondences after Fundamental martix RANSAC')   



gray1 = cv2.cvtColor(img1.copy(),cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2.copy(), cv2.COLOR_BGR2GRAY)

## Compute Dense Disparity

In [ ]:
import numpy as np
import cv2
from skimage.color import rgb2gray, gray2rgb, hsv2rgb
from skimage.transform import ProjectiveTransform

def disp_map(imgL, imgR, block_size=5, search_range=56, lines2):
    
    # Convert images to grayscale
    grayL = rgb2gray(imgL)
    grayR = rgb2gray(imgR)
    
    epipolar_lines = lines2
    
    # Compute image size and block radius
    h, w = grayL.shape
    r = block_size // 2

    ptsL = np.array([(x, y) for y in range(h) for x in range(w)])

    h_disp = np.zeros((h, w))
    v_disp = np.zeros((h, w))
    c_disp = np.zeros((h, w, 3))
    
#     return epipolar_lines
    

    for i, ptL in enumerate(ptsL):

        xL, yL = ptL
        window_left = grayL[max(0, yL-r):min(h, yL+r+1), max(0, xL-r):min(w, xL+r+1)]

        search_min = max(0, xL - search_range)
        search_max = min(w, xL + search_range)

        costs = []
        for x_right in range(search_min, search_max):
            window_right = grayR[max(0, yL-r):min(h, yL+r+1), max(0, x_right-r):min(w, x_right+r+1)]

            window_size = min(window_left.shape[0], window_right.shape[0]), min(window_left.shape[1], window_right.shape[1])
            window_left = window_left[:window_size[0], :window_size[1]]
            window_right = window_right[:window_size[0], :window_size[1]]
            
            #Sum absolute difference
            cost = np.sum(np.abs(window_left - window_right))

            costs.append(cost)

            
        # Find the smallest SAD
        best_match = np.argmin(costs)
        
        #x direction disparity
        disparity = xL - (search_min + best_match)

        
        #Horizontal Disparity
        h_disp[yL, xL] = disparity
        
        #Vertical Disparity
        v_disp[yL, xL] = epipolar_lines[i][0][1]

        # Color Disparity HSV 
        hue = 180 * (np.arctan2(epipolar_lines[i][0][1], disparity) + np.pi) / (2 * np.pi)
        saturation = np.sqrt(disparity**2 + epipolar_lines[i][0][1]**2)
        c_disp[yL, xL] = [hue, saturation, 1]


    # Scale disp maps to 0, 255 range
    h_disp = 255 * (h_disp - np.min(h_disp)) / (np.max(h_disp) - np.min(h_disp))
    v_disp = 255 * (v_disp - np.min(v_disp)) / (np.max(v_disp) - np.min(v_disp))

    return h_disp, v_disp, hsv2rgb(c_disp)



In [ ]:
h_disp, v_disp, c_disp = disp_map(img1, img2, lines2)

cv2.imshow('Disparity Color Component', np.uint8(c_disp))
cv2.imshow('Horizontal Disparity Component', np.uint8(h_disp))
cv2.imshow('Vertical Disparity Component', np.uint8(v_disp))
cv2.waitKey(0)
cv2.destroyAllWindows()